In [6]:
import pandas as pd
import numpy as np
import pyspark 
from pyspark.sql.functions import col
from pyspark.sql import SQLContext

In [7]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

Loading in Instant video reviews and already cleaned comic reviews.

In [3]:
video_reviews_df = spark.read.json('data/reviews_Amazon_Instant_Video.json')
meta_video_reviews = spark.read.json('data/meta_Amazon_Instant_Video.json')
comic_reviews_df = spark.read.json('data/comic_reviews_wtitle.json')

Filtering out the comic reviewers ids and then getting reviews that match.

In [4]:
comic_reviewers = comic_reviews_df.select('reviewerID').distinct().collect()
comic_reviewers = [r[0] for r in comic_reviewers]
video_reviews = video_reviews_df.filter(col('reviewerID').isin(comic_reviewers))
all_video_reviews = video_reviews.select(['asin', 'overall','reviewerID']).toPandas()

Neither title or image information is avaiable in the Instant Video meta, so will just have to mark blank for now. Filling in the counts from dictionary created.

In [12]:
all_video_reviews['title'] = None
all_video_reviews['imUrl'] = 'N/A - Amazon Instant Video'

Adding these to the movie reviews:

In [13]:
movies_df = spark.read.json('data/movie_reviews_wtitle.json').toPandas()
movies_and_video_df = pd.concat([movies_df, all_video_reviews], ignore_index=True, sort=True)

In [14]:
movies_and_video_spark = spark.createDataFrame(
                                movies_and_video_df
                            ).repartition(1).write.json('data/movies_and_video')

_____